In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Load dataset from CSV
df = pd.read_csv('/content/instagram_data_india.csv')

# Debug: Print raw dataframe info
print("Raw DataFrame Info:")
print(df.info())
print(df.head())

# Ensure correct column names
df.columns = [col.strip() for col in df.columns]
print("Column Names After Stripping:", df.columns)

# Function to clean numerical columns
def clean_numeric_column(col):
    return pd.to_numeric(col.replace({'M': '*1e6', '%': ''}, regex=True).map(pd.eval), errors='coerce')

# Convert numerical columns safely
df["FOLLOWERS"] = clean_numeric_column(df["FOLLOWERS"])
df["ER"] = clean_numeric_column(df["ER"])
df["POTENTIAL REACH"] = clean_numeric_column(df["POTENTIAL REACH"])

# Debug: Print before dropping NaNs
print("Data Before Dropping NaN:")
print(df.head())

# Drop rows with missing values
df = df.dropna(subset=["FOLLOWERS", "ER", "POTENTIAL REACH"])

# Debug: Print after dropping NaNs
print("Data After Dropping NaN:")
print(df.head())

# Ensure no empty dataset
if df.empty:
    raise ValueError("Dataset is empty after preprocessing! Check CSV formatting and missing values.")

# Normalize numerical features
features = df[["FOLLOWERS", "ER", "POTENTIAL REACH"]].values
features = (features - np.min(features, axis=0)) / (np.max(features, axis=0) - np.min(features, axis=0))  # Normalize

def parametric_tanh(x, alpha):
    return torch.tanh(alpha * x)

# PPT-LSTM Model Definition
class PPT_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, prob_param=0.5):
        super(PPT_LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.prob_param = prob_param

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.param_alpha = nn.Parameter(torch.tensor(self.prob_param, requires_grad=True))
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        param_activated = parametric_tanh(lstm_out, self.param_alpha)
        output = self.fc(param_activated[:, -1, :])
        return output

input_dim = features.shape[1]
hidden_dim = 10
output_dim = 1
sequence_length = 1
num_samples = features.shape[0]

data = torch.tensor(features, dtype=torch.float32).unsqueeze(1)  # Reshape to (samples, seq_len, features)
targets = torch.rand((num_samples, output_dim))

# Initialize Model
model = PPT_LSTM(input_dim, hidden_dim, output_dim)

# Training Function
def train_ppt_lstm(model, data, targets, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

# Train Model
train_ppt_lstm(model, data, targets)

Raw DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   #                   100 non-null    int64 
 1   NAME                100 non-null    object
 2   FOLLOWERS           100 non-null    object
 3   ER                  100 non-null    object
 4   COUNTRY             100 non-null    object
 5   TOPIC OF INFLUENCE  100 non-null    object
 6   POTENTIAL REACH     100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB
None
   #                             NAME FOLLOWERS     ER COUNTRY  \
0  1         Virat Kohli @virat.kohli    267.1M  0.02%   India   
1  2      Narendra Modi @narendramodi     87.3M  2.23%   India   
2  3         Alia Bhatt 💛 @aliaabhatt     83.7M  0.02%   India   
3  4        Katrina Kaif @katrinakaif     79.7M  0.87%   India   
4  5  दीपिका पादुकोण @deepikapadukone     78.9M  2.18%   Ind